### Let's try some embedding

In [ ]:
!pip3 install tensorflow
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

from keras.utils import plot_model

import pickle
import numpy as np
import csv
from sklearn.metrics import f1_score, confusion_matrix

    100% |████████████████████████████████| 92.5MB 17kB/s 
    100% |████████████████████████████████| 3.2MB 424kB/s 
  Using cached https://files.pythonhosted.org/packages/90/85/64c82949765cfb246bbdaf5aca2d55f400f792655927a017710a78445def/Keras_Applications-1.0.7-py2.py3-none-any.whl
    100% |████████████████████████████████| 368kB 1.4MB/s 
  Using cached https://files.pythonhosted.org/packages/c1/e2/4db8df8f6cddc98e7d7c537245ef2f4e41a1ed17bf0c3177ab3cc6beac7f/numpy-1.16.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 1.2MB 428kB/s 
  Using cached https://files.pythonhosted.org/packages/c0/bf/0315ef6a9fd3fc2346e85b0ff1f5f83ca17073f2c31ac719ab2e4da0d4a3/Keras_Preprocessing-1.0.9-py2.py3-none-any.whl
    100% |████████████████████████████████| 2.2MB 428kB/s 
    100% |████████████████████████████████| 10

In [ ]:
MAX_NB_WORDS = 40000 # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 30 # max length of text (words) including padding
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 200 # embedding dimensions for word vectors (word2vec/GloVe)
GLOVE_DIR = "data/demo/glove.twitter.27B."+str(200)+"d.txt"

In [ ]:
import csv
texts, labels = [], []
print("[i] Reading from csv file...", end="")
with open('data/demo/data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[0])
        labels.append(row[1])
print("Done!")

In [ ]:
len(texts)

Dataset has been labeled on 5 emotions: "Neutral", "Happy", "Sad", "Hate", "Anger"

In [ ]:
texts

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
with open('tokenizerv1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("[i] Saved word tokenizer to file: tokenizerv1.pickle")

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('[i] Found %s unique tokens.' % len(word_index))
data_int = pad_sequences(sequences, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
data = pad_sequences(data_int, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))

In [ ]:
from keras.utils.np_utils import to_categorical
labels = to_categorical(np.asarray(labels)) # convert to one-hot encoding vectors
print('[+] Shape of data tensor:', data.shape)
print('[+] Shape of label tensor:', labels.shape)

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [ ]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [ ]:
embeddings_index = {}
f = open(GLOVE_DIR,  encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# static channel
embedding_layer_frozen = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
embedded_sequences_frozen = embedding_layer_frozen(sequence_input)

In [ ]:
lstm_1 = (Bidirectional(LSTM(32, return_sequences = True)))(embedded_sequences_frozen)
pool_1 = (GlobalMaxPool1D())(lstm_1)
dense_1 = (Dense(20, activation="relu"))(pool_1)
dropout_1 = (Dropout(0.05))(dense_1)
preds = (Dense(5, activation="softmax"))(dropout_1)

advanced_model = Model(sequence_input, preds)
advanced_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 10
advanced_model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
y_pred_adv = advanced_model.predict(x_val)
pred_adv = np.argmax(y_pred_adv, axis=1)
real_adv = np.argmax(y_val, axis=1)
print('F1-score: {0}'.format(f1_score(pred_adv, real_adv, average='micro')))
print('Confusion matrix:')
confusion_matrix(pred_adv, real_adv)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
classes = ["neutral", "happy", "sad", "hate","anger"]
print(classification_report(pred_adv, real_adv, target_names=classes))

In [ ]:
def predict_emotion(text):
    sequences = tokenizer.texts_to_sequences([text])
    word_index = tokenizer.word_index
    data_int = pad_sequences(sequences, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
    data = pad_sequences(data_int, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))
    prediction_ar = advanced_model.predict(data)
    pr = np.argmax(prediction_ar, axis=1)
    print(prediction_ar)
    if pr == 0:
        return "Neutral"
    if pr == 1:
        return "Happy"
    if pr == 2:
        return "Sad"
    if pr == 3:
        return "Hate"
    if pr == 4:
        return "Anger"
    return pr

print(predict_emotion("What a lovely day! What a lovely day! What a lovely day!"))
print(predict_emotion("I hate you"))
print(predict_emotion("A king in the castle"))
print(predict_emotion("i am so sorry for what happened!"))
print(predict_emotion("I'm angry of you!"))

print(predict_emotion("The program normally began to work only in the morning!!"))

In [ ]:
print(predict_emotion("""When you were here before
Couldn't look you in the eye
You're just like an angel
Your skin makes me cry
You float like a feather
In a beautiful world
And I wish I was special
You're so fuckin' special
But I'm a creep, I'm a weirdo.
What the hell am I doing here?
I don't belong here.
I don't care if it hurts
I want to have control
I want a perfect body
I want a perfect soul
I want you to notice
When I'm not around
You're so fuckin' special
I wish I was special
But I'm a creep, I'm a weirdo.
What the hell am I doing here?
I don't belong here.
She's running out again,
She's running out
She's run run run run
Whatever makes you happy
Whatever you want
You're so fuckin' special
I wish I was special
But I'm a creep, I'm a weirdo,
What the hell am I doing here?
I don't belong here.
I don't belong here."""))